In [1]:
from pathlib import Path
import json
import re

import torch
from torch import Tensor
import torch.optim as optim
from torch.utils.data import Dataset
from torch import nn
from torch import functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import image as mpl_image
from typing import Type, Any, Callable, Union, List, Optional

from hmmlearn import hmm


In [2]:
batch_size = 4

classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5}

In [3]:
test_folder = Path("/nic/test")
model_file = Path("/nic/models/model_20.pyt")
annotations_file = Path("/nic/annotations.json")
sequences_file = Path("/nic/sequences.npy")
emission_matrix_path = Path("/nic/emission_matrix.npy")


In [4]:
[x for x in test_folder.rglob("*.png")]

[WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[101]_frame[0].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[16].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[4].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[6].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[9].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[11].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[1].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[2].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[3].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02

In [5]:
class ImageDataset(Dataset):
    def __init__(self, dataset_folders_path, image_transform, target_transform):
        
        self.dataset_folders_path = dataset_folders_path
        
        self.data = {}
        for path in dataset_folders_path.rglob("*.png"):
            label = path.parts[-2]
            image = mpl_image.imread(str(path))#torchvision.read_image(path)
            self.data[str(path)] = {"image": image, "label": label}
            
        self.keys = list(self.data.keys())
        
        self.image_transform = image_transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        sample_key = self.keys[idx]
        sample = self.data[sample_key]
        image = self.image_transform(sample["image"])
        label = self.target_transform(sample["label"])
#         print(sample)
        
        return {"image": image, "label": label, "path": str(sample_key)}

In [6]:
class BlindImageDataset(Dataset):
    def __init__(self, dataset_folders_path, image_transform, target_transform):
        
        self.dataset_folders_path = dataset_folders_path
        
        self.data = {}
        for path in dataset_folders_path.rglob("*.png"):
            label = "Unidentified"
            image = mpl_image.imread(str(path))#torchvision.read_image(path)
            self.data[str(path)] = {"image": image, "label": label}
            
        self.keys = list(self.data.keys())
        
        self.image_transform = image_transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        sample_key = self.keys[idx]
        sample = self.data[sample_key]
        image = self.image_transform(sample["image"])
        label = self.target_transform(sample["label"])
#         print(sample)
        
        return {"image": image, "label": label, "path": str(sample_key)}

In [7]:
def label_to_tensor(label, classes):
    index = classes[label]

    return index

In [8]:
image_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        torchvision.transforms.Grayscale(num_output_channels=1),
        torchvision.transforms.Normalize(0.0, 1.0, inplace=False),
        torchvision.transforms.RandomAffine(
            (-180.0, 180.0), 
            translate=(0.25,0.25),
            interpolation=transforms.functional.InterpolationMode.BILINEAR,
        ),
        torchvision.transforms.GaussianBlur(
            (3,3), 
            sigma=(0.01, 2.0),
        ),
    ]
)



target_transform = lambda x: label_to_tensor(x, classes)


In [9]:
testset = BlindImageDataset(test_folder, image_transform, target_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, 
                                        # num_workers=1
                                         drop_last=True
                                        )

# Define the model

In [10]:

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 6,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
#         print(x.shape)
        x = self.layer2(x)
#         print(x.shape)
        x = self.layer3(x)
#         print(x.shape)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [11]:
def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [12]:
def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

# Load the model

In [13]:
model = resnet18()


In [14]:
model.load_state_dict(torch.load(str(model_file)))


<All keys matched successfully>

# Annotate the data with the model

In [15]:
confusion_matrix_test = np.zeros((len(classes), len(classes)))
annotations = {}
for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
    path = data["path"]
    inputs = data["image"]
    labels = data["label"]
    
    
    # forward + backward + optimize
    outputs = model(inputs)


    
    for j in [0,1,2,3]:
        confusion_matrix_test[labels[j], torch.argmax(outputs[j])] += 1
        label = int(torch.argmax(outputs[j]).detach().cpu())
        annotations[path[j]] = label
        

# Output the confusion matrix

In [16]:
confusion_matrix_test_table = pd.DataFrame(
data=confusion_matrix_test,
index=list(classes.keys()),
columns=list(classes.keys()))

In [17]:
confusion_matrix_test_table

,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified
Not_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Precieved_Not_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Precieved_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Slanted,0.0,0.0,0.0,0.0,0.0,0.0
Unidentified,650.0,433.0,692.0,499.0,1071.0,63.0


# Save to json

In [18]:
annotations

{'\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[101]_frame[0].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[16].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[4].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[6].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[9].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[11].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[1].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[2].png': 4,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[3].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[4].png': 0,
 '\\nic\\test\\Not

In [19]:
new_annotations = {}

for path, annotation in annotations.items():
    regex = "_particle\[([0-9]+)\]_frame\[([0-9]+)\]"
    capture = re.findall(regex, path)
    particle = int(capture[0][0])
    frame = int(capture[0][1])
    try:
        new_annotations[particle][frame] = annotation
    except:
        new_annotations[particle] = {}
        new_annotations[particle][frame] = annotation

In [21]:
sequences = {}
ordered_annotations = {}

k = 0
sequence = []
for particle in new_annotations:
    ordered_annotations[particle] = []
    
    for j in range(20):
        try:
            next_annotation = new_annotations[particle][j]
            sequence.append(next_annotation)
        except:
            sequence.append(6)
    sequences[k] = sequence
    k = k + 1
    sequence = []
    

In [22]:
sequences

{0: [2, 4, 1, 4, 4, 3, 3, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 1, 3, 1],
 1: [0, 2, 0, 4, 0, 2, 0, 4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2, 4],
 2: [0, 0, 4, 0, 0, 4, 4, 4, 2, 4, 4, 2, 4, 5, 4, 4, 2, 4, 2, 4],
 3: [0, 4, 4, 4, 4, 3, 4, 1, 4, 2, 4, 4, 4, 2, 4, 3, 4, 2, 5, 2],
 4: [0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 2, 2, 0, 2, 2, 2],
 5: [2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 4],
 6: [0, 0, 0, 0, 4, 2, 2, 0, 4, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0],
 7: [2, 2, 0, 2, 2, 4, 4, 4, 4, 4, 2, 2, 0, 0, 2, 4, 0, 4, 2, 2],
 8: [0, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 0, 0, 0, 0, 4, 0, 2, 4, 2],
 9: [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 4, 2, 0],
 10: [0, 2, 0, 2, 0, 0, 2, 2, 2, 4, 0, 2, 4, 0, 0, 2, 4, 0, 2, 2],
 11: [3, 1, 4, 3, 4, 4, 4, 4, 4, 0, 0, 2, 2, 5, 6, 2, 4, 4, 4, 4],
 12: [0, 2, 4, 4, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 4, 4, 2, 5],
 13: [0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 4, 4, 2, 2, 2, 2],
 14: [4, 0, 0, 2, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
 15: 

In [23]:
full_sequences = [sequence for sequence in sequences.values() if len(sequence) == 20]

In [24]:
sequences_array = np.array(list(sequences.values()))
sequences_array.shape

(172, 20)

In [25]:
np.save(str(sequences_file), sequences_array)

In [26]:
naive_transition_matrix = np.zeros((7,7))
for sequence in sequences.values():
    prev_state = 6
    for state in sequence:
        naive_transition_matrix[prev_state, state] += 1
        prev_state = state


array([[329.,   1., 193.,   1., 106.,   1.,   3.],
       [  0., 151.,   7., 158.,  96.,   5.,   0.],
       [191.,   7., 205.,  14., 220.,   9.,   5.],
       [  1., 149.,  13., 180., 119.,   8.,   3.],
       [ 83.,  80., 235., 126., 461.,  25.,   4.],
       [  2.,   5.,   9.,   6.,  21.,  11.,   1.],
       [ 44.,  40.,  30.,  14.,  48.,   4.,  16.]])

In [28]:

classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
naive_transition_matrix = naive_transition_matrix / np.sum(naive_transition_matrix, axis=1).reshape((-1,1))
print(np.sum(naive_transition_matrix, axis=1))

confusion_matrix_test_table = pd.DataFrame(
data=naive_transition_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table 

[1. 1. 1. 1. 1. 1. 1.]


,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified,No_sample
Not_Oriented,0.518927,0.001577,0.304416,0.001577,0.167192,0.001577,0.004732
Oriented,0.000000,0.362110,0.016787,0.378897,0.230216,0.011990,0.000000
Precieved_Not_Oriented,0.293395,0.010753,0.314900,0.021505,0.337942,0.013825,0.007680
Precieved_Oriented,0.002114,0.315011,0.027484,0.380550,0.251586,0.016913,0.006342
Slanted,0.081854,0.078895,0.231755,0.124260,0.454635,0.024655,0.003945
Unidentified,0.036364,0.090909,0.163636,0.109091,0.381818,0.200000,0.018182
No_sample,0.224490,0.204082,0.153061,0.071429,0.244898,0.020408,0.081633


# Hidden Markov model

In [30]:
emission_matrix_np = np.load(emission_matrix_path)

NameError: name 'emission_matrix_path' is not defined

In [ ]:
model.emissionprob_ = emission_matrix_np

In [ ]:
model = hmm.MultinomialHMM(n_components=7, n_iter=n_iter, params="st", init_params="st")

In [ ]:
model.fit(sequences.reshape(-1, 1), lengths)

In [ ]:
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
confusion_matrix_test_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table